# Présentation

Blabla

# Sommaire

### [Bibliothèques](#1_bibli)
### [Fonctions](#1_funcs)
### [Données](#1_donnees)
### [Profiling](#1_profiling)
### [Rangement des colonnes](#1_rangement)
<!-- ### [Idée](#1_idee)
### [Nettoyage](#1_nettoyage)
- [Labels](#nett_1)
- [Categories - Complétion de données par ML](#nett_2)
- [Packaging](#nett_3)

### [Analyses Univariées](#1_univ)
- [Labels](#univ_1)
- [Categories](#univ_2)
- [Packaging](#univ_3)
- [Additifs](#univ_4)
- [Huile de palme](#univ_5)
- [Graisse saturée](#univ_6)
- [Nutriscore numérique](#univ_7)
- [Nutriscore catégorique](#univ_8)

### [Analyses Bivariées](#1_biv)
- [Entre l'emballage et le Nutriscore (Score)](#biv_emb_nutr_score)
- [Entre l'emballage et le Nutriscore (Grade)](#biv_emb_nutr_grade)
- [Entre l'emballage et le nombre d'additifs](#biv_emb_add)
- [Entre l'emballage et les labels](#biv_emb_label)

### [Analyse Statistique](#1_anal_stat)
### [Tests Statistique](#1_tests_stat)
### [Conclusion](#1_conclusion) -->

<a id='1_bibli'></a>

# Import de bibliothèques 📚

In [19]:
import pandas as pd
pd.set_option("mode.chained_assignment", None)
from pandas_profiling import ProfileReport

import numpy



<a id='1_funcs'></a>

# Fonctions ⚙️

In [20]:
# def plot_histo(data, title=None, xlabel=None, drop_outliers=False, bins='sturges', plot_density=True):
#     """Plots an histogram from an array of numbers. Bins can be set or automatically calculated by sturges method
#     It's also possible to plot the density curve an drop outliers"""
#     if drop_outliers:
#         data = data[data.between(data.quantile(drop_outliers), data.quantile(1 - drop_outliers))]
#     if bins == 'sturges':
#         bins = int(1+log2(len(data)))
#     plt.style.use("seaborn")
#     if plot_density:
#         density = gaussian_kde(data)
#         xs = np.linspace(data.min(),data.max(),200)
#         density.covariance_factor = lambda : .25
#         density._compute_covariance()
#         plt.plot(xs,density(xs), label="Courbe de densité")
#         plt.legend()
#     plt.ylabel("frequency")
#     plt.xlabel(xlabel)
#     plt.title(title)
#     plt.hist(data, bins=bins, density=True)
#     plt.show()

In [21]:
# def split_custom(label_string, delimiter=','):
#     """splits a string into a list"""
#     return label_string.split(delimiter)

In [22]:
# def get_unique_elements(panda_serie):
#     """This functions returns the list of unique elements 
#     out of a panda serie of strings with comma separated elements"""
#     all_labels = set()
#     for labels in panda_serie.map(split_custom):
#         for label in labels:
#             all_labels.add(label)
#     return list(all_labels)

<a id='fonctionbio'></a>

In [23]:
# def test_bio(labels):
#     """From a list of labels this function returns a booleean if 'organic' or 'bio' is in one of the labels"""
#     bool_test = False
#     for label in labels:
#         label = label.lower()
#         if 'organic' in label or 'bio' in label:
#             bool_test = True
#             break
#     return bool_test

Retour au [script](#backbio)

<a id='fonctionvege'></a>

In [24]:
# def test_vege(labels):
#     """From a list of labels this function returns a booleean if 'vegeterian' or 'vegan' is in one of the labels"""
#     bool_test = False
#     for label in labels:
#         label = label.lower()
#         if 'vegetarian' in label or 'vegan' in label:
#             bool_test = True
#             break
#     return bool_test

Retour au [script](#backvege)

<a id='fonctionnegation'></a>

In [25]:
# # # This function works but would be more efficient if it would detect the language. We'll save that for later
# def test_no_something(labels, dict_something):
#     """From a list of labels this function returns a booleean if 'preserative' and a negation is in one of the labels"""
#     for word, negations in dict_something.items():
#         for label in labels:
#             label = label.lower()
#             label = label.replace('-', ' ')
#             bool_list = []
#             #looking for word preservative
#             preservative_in_label = bool(re.match(r'.*\b{}.*\b'.format(word), label))
#             if preservative_in_label:
#                 #looking for negation
#                 for negation in negations:
#                     if bool(re.match(r'.*\b{}\b'.format(negation), label)):
#                         return True

#     return False

- Retour au [script](#backconserv) Conservateurs
- Retour au [script](#backcolor) Colorants
- Retour au [script](#backGMO) OGMs

<a id='plot_bar'></a>

In [26]:
def plot_bar(data, labels, color='purple', label=None, rotation=False, ylabel='Proportion par rapport au nombre de poduits du dataset (%)'):
    """Custom function to plot a bars"""
    plt.figure(figsize=(8,8))
    plt.style.use('seaborn')
    plt.bar(list(range(1, len(data)+1)), data, color=color, label=label)
    if rotation:
        angle=90
    else:
        angle=0
    plt.xticks(list(range(1, len(data)+1)), labels, rotation=angle, fontsize=12)
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()

- Retour au [script](#univ_1_plot_1) Labels
- Retour au [script](#univ_1_plot_2) Categories

<a id='launch_kmeans_mlflow'></a>

In [27]:
# def launch_kmeans_mlflow(data_kmeans, n_clusters, comment, col_target, max_iter=300, n_init=10, init='k-means++', random_state=41):
#     """Trains a KMeans model and saves the results into MLFLOW"""
#     with mlflow.start_run(run_name=comment):
        
#         #Train
#         start = time.time()
#         kmeans = KMeans(n_clusters=n_clusters, max_iter=max_iter, n_init=n_init, init=init, random_state=random_state).fit(data_kmeans.drop(columns=[col_target]))
#         elapsed = time.time() - start
        
#         #Predict
#         data_kmeans['predicted_group'] = kmeans.predict(data_kmeans.drop(columns=[col_target]))
        
#         #Undestang predictions
#         categs_group_number = {}
#         for i in range(n_clusters):
#             categs_group_number[i] = data_kmeans[data_kmeans['predicted_group'] == i][col_target].value_counts().index[0]
        
#         data_kmeans = data_kmeans.merge(pd.Series(categs_group_number, name='predicted_group_label'), left_on='predicted_group', right_index=True)
        
#         #Test
#         data_kmeans['model_is_right'] = data_kmeans[col_target] == data_kmeans['predicted_group_label']
        
#         perf = data_kmeans.model_is_right.value_counts(normalize=True).loc[True]
        
#         mlflow.log_param("n_clusters", n_clusters)
#         mlflow.log_param("max_iter", max_iter)
#         mlflow.log_param("n_init", n_init)
#         mlflow.log_param("init", init)
#         mlflow.log_param("random_state", random_state)
        
#         mlflow.log_metric("accuracy", perf)
#         mlflow.log_metric("distinct_categs_predicted", len(data_kmeans.predicted_group_label.unique()))
#         mlflow.log_metric("trainig_time", elapsed)
#         mlflow.log_metric("inertie", kmeans.inertia_)
#         mlflow.log_metric("nb_iter", kmeans.n_iter_)

        
#         mlflow.set_tag("columns_passed", str(data_kmeans.drop(columns=[col_target, 'predicted_group', 
#                                                                       'predicted_group_label']).columns.values))
#         mlflow.end_run()
#     return data_kmeans

- Retour au [script](#categ_kmeans) K-means

<a id='launch_gb_mlflow'></a>

In [28]:
# def launch_gb_mlflow(data, comment, col_target, n_estimators=50, learning_rate=0.2,  max_depth=5, random_state=41):
#     """Trains a GBoost model and saves the results into MLFLOW"""
#     data_train, data_test = train_test_split(data, train_size=0.7, random_state=random_state)
#     with mlflow.start_run(run_name=comment):
#         start_time = time.time()
#         gb_cl = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate,  max_depth=max_depth, random_state=random_state).fit(data_train.drop(columns=[col_target]), data_train[col_target])
#         elapsed = time.time() - start_time
        
#         mlflow.log_param("n_estimators", n_estimators)
#         mlflow.log_param("learning_rate", learning_rate)
#         mlflow.log_param("max_depth", max_depth)
#         mlflow.log_param("random_state", random_state)
        
#         mlflow.log_metric("score sur train", gb_cl.score(data_train.drop(columns=[col_target]), data_train[col_target]))
#         mlflow.log_metric("score sur test", gb_cl.score(data_test.drop(columns=[col_target]), data_test[col_target]))
#         mlflow.log_metric("time_train", elapsed)
        
#         mlflow.end_run()
    
#     data_result = pd.DataFrame({"actual": data_test[col_target], "predicted" : gb_cl.predict(data_test.drop(columns=[col_target]))})
    
#     return (data_result, gb_cl)

- Retour au [script](#categ_gb) Gradient boosting

<a id='parser_prediction'></a>

In [29]:
# def parser_prediction(row, data_unknown, col_modeled, col_predicted):
#     """Completes unknown values in a column from data in another dataset"""
#     if row[col_modeled] == 'unknown':
#         return data_unknown.loc[row.name][col_predicted]
#     else:
#         return row[col_modeled]

- Retour au [script](#back_parse_gb_results) 

<a id='1_donnees'></a>

# Données 🎁

In [30]:
data_2016 = pd.read_csv('data/2016-building-energy-benchmarking.csv', sep=',')
data_2015 = pd.read_csv('data/2015-building-energy-benchmarking.csv', sep=',')

In [31]:
data_2016.shape

(3376, 46)

In [32]:
data_2015.shape

(3340, 47)

In [33]:
data = pd.concat([data_2015, data_2016])

In [34]:
data.shape

(6716, 56)

On remarque ici un problème de format des colonnes entre les données 2015 et 2016. Il faut retrouver les correspondances entre les colonnes pour obtenir un dataset robuste

<a id='1_profiling'></a>

# Profiling 🔍

Etant données les dimensions de ce dataset, je vais réaliser l'analyse préliminaire avec Pandas Profiling. L'analyse est exportée en HTML pour une meilleure lisibilité du NB

In [35]:
profile = ProfileReport(data, title="Report on Seattle Building Benchmarking", minimal=True)

In [36]:
profile.to_file('profile_report.html')

Summarize dataset:   0%|          | 0/66 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Voir le report : [cliquer ici](profile_report.html)

<a id='1_rangement'></a>

# Rangement des colonnes 🗂️